### Imports

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../Utilities')
from tqdm import tqdm
import warnings
import time as t
import os
import cv2
import glob
import argparse
from os.path import isfile, join
from PIL import Image
import multiprocessing as mp
from PIL import ImageDraw
from PIL import ImageFont
from itertools import groupby
from operator import itemgetter
warnings.filterwarnings("ignore")
import pickle

### Load and Process Dataframe

In [2]:
filename = '../Files/optical_flow'
infile = open(filename,'rb')
df = pickle.load(infile)
infile.close()

In [3]:
df['dataset_name'].unique()

array(['CAF26', 'CAF34', 'CAF42', 'CAF69', 'EAB40', 'EAB50_5state',
       'CAF99', 'CAF106', 'CAF77'], dtype=object)

In [4]:
caf26_df = df[df['dataset_name'] == 'CAF26']
caf34_df = df[df['dataset_name'] == 'CAF34']
caf42_df = df[df['dataset_name'] == 'CAF42']
caf69_df = df[df['dataset_name'] == 'CAF69']
eab40_df = df[df['dataset_name'] == 'EAB40']
eab50_df = df[df['dataset_name'] == 'EAB50_5state']
caf99_df = df[df['dataset_name'] == 'CAF99']
caf106_df = df[df['dataset_name'] == 'CAF106']
caf77_df = df[df['dataset_name'] == 'CAF77']

### Set Up File Paths

In [5]:
os.chdir(os.path.dirname(os.getcwd()))

In [6]:
cwd_path = os.getcwd()
documents_path = '/Users/graciezhang/Documents/Hengen-Lab'
external_path = f'{documents_path}/Hengen-Lab-Work'
buffer_path = f'{documents_path}/Hengen-Lab-Work/buffer'
put_path = f'{documents_path}/Hengen-Lab-Work/Notebooks/video_clipper/Mice_Clips'
remote_path = "/media/HlabShare/Aidan_Work/flicker_videos/"

In [7]:
# 77 is the 56 gig video
fps_animals = {"CAF99": 30,"CAF106": 30,"CAF26":15, "EAB40": 15, "EAB50": 15, "CAF69": 30, "CAF42": 15, "CAF34": 15, "CAF77" : 15}

### Open Video and Check Video

In [8]:
#buffer directory outside directory of notebook
def convert_frames_to_video(pathIn, pathOut, fps, mouse_name):
    frame_array = []
    os.chdir(cwd_path)
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    
    os.chdir(buffer_path)
    if len(files) <= 1: return
    
    files = np.sort([int(i[:-4]) for i in files])
    files = [str(i)+'.jpg' for i in files]
    
    for i in range(1,len(files)):
        filename= files[i]
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        frame_array.append(img)
        
    os.chdir(put_path)
    cur_dir = os.getcwd()
    out = cv2.VideoWriter(cur_dir + f'/{mouse_name}' + pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    
    for i in range(len(frame_array)): out.write(frame_array[i])
    out.release()

In [9]:
def flick(flicker):
    if flicker == None: return 'None'
    elif flicker == 'WAKE_WITHIN_NREM': return 'WN'
    elif flicker == 'REM_WITHIN_NREM': return 'RN'
    elif flicker == 'NREM_WITHIN_REM': return 'NR'
    elif flicker == 'NREM_WITHIN_WAKE': return 'NW'
    elif flicker == 'REM_WITHIN_WAKE': return 'RW'
    elif flicker == 'WAKE_WITHIN_REM': return 'WR'

def write_text_frame(frame_name,sleep_state,pred_sleep_state,flicker):
    img = Image.open(frame_name)
    drawer = ImageDraw.Draw(img)
    os.chdir(cwd_path)
    font_path = documents_path+'/Hengen-Lab-Work/Notebooks/video_clipper/SpaceMono-Regular.ttf'
    font = ImageFont.truetype(font_path, 15)
    os.chdir(buffer_path)
    drawer.text((img.size[0]*0.7, img.size[1]*0.15), f"Sleep State: {num_sleep_conv(sleep_state)}", fill=(255,255,255),font=font)
    drawer.text((img.size[0]*0.7, img.size[1]*0.25), f"Flicker Type: {flick(flicker)}", fill=(255,255,255),font=font)
    drawer.text((img.size[0]*0.7, img.size[1]*0.2), f"Pred Sleep State: {num_sleep_conv(pred_sleep_state)}", fill=(255,255,255),font=font)
#     if type(behavior) == list:
#         loc_behavior = (num_cat(behavior[0])) if behavior[0] < 10 else 'None'
#         reg_behavior = 'None' if behavior[1] < 10 else (num_cat(behavior[1]))
#     else:
#         loc_behavior = (num_cat(behavior)) if behavior < 10 else 'None'
#         reg_behavior = 'None' if behavior < 10 else (num_cat(behavior))
#     drawer.text((img.size[0]*0.7, img.size[1]*0.25), f"LoConst: None", fill=(255,255,255),font=font)
#     drawer.text((img.size[0]*0.7, img.size[1]*0.3), f"Regular: None", fill=(255,255,255),font=font)
    img.save(frame_name)
    

### Do all  Videos for an animal

In [10]:
def num_sleep_conv(state): return 'W' if state == 0 else ('N' if state == 1 else 'R')

In [11]:
def num_flick(flicker):
    if flicker == 0: return 'None'
    elif flicker == 1: return 'NW'
    elif flicker == 2: return 'RW'
    elif flicker == 3: return 'NR'
    elif flicker == 4: return 'WR'
    elif flicker == 5: return 'RN'
    elif flicker == 6: return 'WN'


In [12]:
def num_cat(behavior):
    if behavior == 1: return 'Nest'
    elif behavior == 2: return 'PBall'
    elif behavior == 3: return 'PBox'
    elif behavior == 4: return 'RBall'
    elif behavior == 5: return 'RBox'
    elif behavior == 6: return 'Drink'
    elif behavior == 7: return 'Chew'
    elif behavior == 8: return 'Port'
    elif behavior == 9: return 'Groom'
    elif behavior == 10: return 'CFront'
    elif behavior == 11: return 'CPort'
    elif behavior == 12: return 'Dig'
    elif behavior == 13: return 'Motion'
    elif behavior == 14: return 'MPM'
    elif behavior == 15: return 'Rear'
    elif behavior == 16: return 'Stretch'
    elif behavior == 17: return 'Twitch'

In [13]:
def flushBuffer(): # flush all frames from folder
    dir = buffer_path
    for f in os.listdir(buffer_path):
        os.remove(os.path.join(buffer_path, f))

In [14]:
def flushClips(mouse_name): # flush all frames from folder
    dir = f'{put_path}/{mouse_name}'
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))

In [15]:
def find_groups(flickers):
    flickers = sorted(set(flickers))
    gaps = [[s, e] for s, e in zip(flickers, flickers[1:]) if s+1 < e]
    edges = iter(flickers[:1] + sum(gaps, []) + flickers[-1:])
    return list(zip(edges, edges))

def create_flicker_clips(df,animal):
    #start_frame = 1
    flickers = np.array(np.nonzero(np.array(df.flicker_flavor))[0])
    flicker_groups = find_groups(flickers) # 15 fps, 54000
    
    os.chdir(put_path)
    
    os.chdir(cwd_path)
    
    for group in flicker_groups:
        start_frame = group[0]
        end_frame = group[1]
        original_start = start_frame # 15 fps, 54000
        original_end = end_frame # 15 fps, 54000
        # df idx locations
        start_frame_idx = int(((original_start+original_end)/2)-(30*15)) # df idx
        end_frame_idx = int(((original_start+original_end)/2)+(30*15)) # df idx location
        idx_locations = np.arange(start_frame_idx, end_frame_idx+1, 1)
        if idx_locations[-1] > 54000 or idx_locations[0] < 0 or idx_locations[-1] < 0 or idx_locations[0] > 54000: continue
        # vid idx locations
        vid_start_idx = int(start_frame_idx*(fps_animals[animal]/15)-1) if fps_animals[animal] != 15 else start_frame_idx
        vid_end_idx = int(end_frame_idx*(fps_animals[animal]/15)) if fps_animals[animal] != 15 else end_frame_idx
        i = 0
        idx = 0
        if df.iloc[start_frame].flicker_flavor == 'WAKE_WITHIN_NREM':
            for frame in tqdm(range(vid_start_idx,vid_end_idx-1 if fps_animals[animal] !=15 else vid_end_idx)): # vid location
                os.chdir(buffer_path)
                cap.set(1, frame)
                ret, still = cap.read()
                frame_name = f'{frame}.jpg'
                cv2.imwrite(frame_name, still)
                flicker = df.iloc[idx_locations[idx]].flicker_flavor
                sleep_state = df.iloc[idx_locations[idx]].label_wnr_012
                pred_sleep_state = df.iloc[idx_locations[idx]].predicted_wnr_012
                write_text_frame(frame_name,sleep_state,pred_sleep_state,flicker)
                if fps_animals[animal] != 15: 
                    if i % (fps_animals[animal]/15) == 1: idx+=1
                else: idx+=1
                i+=1
                
            os.chdir(buffer_path)
            
            ds_store_file_location = f'{buffer_path}/.DS_Store'
            if os.path.isfile(ds_store_file_location):
                os.remove(ds_store_file_location)

            
            convert_frames_to_video(buffer_path, f'/{video_name}_f{original_start}-f{original_end}.mp4',fps_animals[animal], animal)
            
            os.chdir(buffer_path)
            
            flushBuffer()

In [ ]:
for video in caf106_df.video_filename.unique():
    cur_df = caf106_df[caf106_df['video_filename'] == video]
    mouse_name = 'CAF106'
    os.chdir(external_path)
    cur_dir = os.getcwd()
    video_path = f'{cur_dir}/videos/{mouse_name}/{video}'
    cap = cv2.VideoCapture(video_path)
    video_name = video
    cur_df = df[df['video_filename'] == video_name]
    video_name = video_name[:len(video_name)-4]
    cur_df = df[df['video_filename'] == video]
    create_flicker_clips(cur_df, mouse_name)

100%|███████████████████████████████████████| 1800/1800 [01:45<00:00, 16.99it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [01:58<00:00, 15.25it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [01:51<00:00, 16.09it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [01:59<00:00, 15.11it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████

100%|███████████████████████████████████████| 1800/1800 [01:54<00:00, 15.76it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [01:58<00:00, 15.14it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [02:05<00:00, 14.34it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [02:33<00:00, 11.74it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████

100%|███████████████████████████████████████| 1800/1800 [02:31<00:00, 11.85it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [02:24<00:00, 12.47it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [02:16<00:00, 13.18it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [02:18<00:00, 12.96it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████

100%|███████████████████████████████████████| 1800/1800 [04:24<00:00,  6.81it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [01:56<00:00, 15.42it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [04:46<00:00,  6.28it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████████████████████████████████████| 1800/1800 [03:09<00:00,  9.48it/s]
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|███████

In [24]:
flushBuffer()

In [123]:
flushClips('CAF42')